In [1]:
import time
from functools import wraps
from inspect import signature

def timethis(func):
    '''Декоратор, который выводит время выполнения.'''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result 
    return wrapper

@timethis
def countdown(n: int):
    '''Counts down'''
    while n > 0:
        n -= 1
        

countdown(100000)
countdown(10000000)
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)
print(signature(countdown))

countdown 0.004997730255126953
countdown 0.4742562770843506
countdown
Counts down
{'n': <class 'int'>}
(n: int)


In [2]:
def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def add(x, y):
    return x + y

print(add(2, 3))
print(add.__wrapped__(2, 3))

Decorator 1
Decorator 2
5
Decorator 2
5


In [3]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    '''
    Добавляет логирование в функцию. level – это уровень логирования,
    name – это название логгера, message – это сообщение в лог. Если
    name и message не определены, они будут дефолтными от имени функции
    и ее модуля.
    '''

    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate


# Пример использования
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')
    

spam()   
print(add(2, 3))    

spam


Spam!
5


In [4]:
from functools import wraps, partial
import logging

# Вспомогательный декоратор для прикрепления
# к функции в качестве атрибута obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    '''
    Добавляет логирование в функцию. level – это уровень логирования,
    name – это название логгера, message – это сообщение в лог. Если
    name и message не определены, они будут дефолтными от имени функции
    и ее модуля.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        # Прикрепляем функции-сеттеры
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
        
        return wrapper
    return decorate


# Пример использования
@logged(logging.DEBUG)
def add(x, y):
    return x + y
    
@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')
    
logging.basicConfig(level=logging.DEBUG)
print(add(2, 3))

# Изменение сообщения в лог
add.set_message('Add called')
print(add(2, 3))

# Изменение уровня логирования
add.set_level(logging.WARNING)
print(add(2, 3))

DEBUG:__main__:add
DEBUG:__main__:Add called


5
5
5


In [5]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # Если мы в оптимизированном режиме, отключаем проверку типов
        if not __debug__:
            return func
        
        # Отображаем имена аргументов функции на предоставленные типы
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            
            # Принудительно проверяем типы предоставленных аргументов ассертами
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(f'Argument {name} must be {bound_types[name]}')
    
            return func(*args, **kwargs)
        return wrapper
    return decorate


@typeassert(int, float)
def add(x, y):
    return x + y

add(2, 3.5)

5.5

In [6]:
from functools import wraps

class A:
    # Декоратор как метод экземпляра
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper
    
    # Декоратор как метод класса
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper
    

# Как метод экземпляра
a = A()

@a.decorator1
def spam():
    pass

# Как метод класса
@A.decorator2
def grok():
    pass

spam()
grok()

Decorator 1
Decorator 2


In [8]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
    
    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)
        

@Profiled
def add(x, y):
    return x + y


class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)
        
print(add(2, 3))
print(add.ncalls)

s = Spam()
s.bar(1)        
s.bar(2)        
Spam.bar.ncalls                 

5
1
<__main__.Spam object at 0x0000016B9087D8D0> 1
<__main__.Spam object at 0x0000016B9087D8D0> 2


2

In [9]:
from functools import wraps

def profiled(func):
    ncalls = 0
    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal ncalls
        ncalls += 1
        return func(*args, **kwargs)
    wrapper.ncalls = lambda: ncalls
    return wrapper

# Пример
@profiled
def add(x, y):
    return x + y

print(add(2, 3))
print(add(4, 5))
print(add.ncalls())

5
9
2


In [10]:
import time
from functools import wraps

# Простой декоратор
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(end-start)
        return r
    return wrapper

# Класс, иллюстрирующий применение декоратора к различным типам методов
class Spam:
    @timethis
    def instance_method(self, n):
        print(self, n)
        while n > 0:
            n -= 1

    @classmethod
    @timethis
    def class_method(cls, n):
        print(cls, n)
        while n > 0:
            n -= 1

    @staticmethod
    @timethis
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1
            
            
s = Spam()
s.instance_method(1000000)
Spam.class_method(1000000)
Spam.static_method(1000000)                   

<__main__.Spam object at 0x0000016B9087DEA0> 1000000
0.03229975700378418
<class '__main__.Spam'> 1000000
0.03539919853210449
1000000
0.03390622138977051


In [11]:
from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)
    return wrapper


@optional_debug
def spam(a,b,c):
    print(a,b,c)
    
    
spam(1,2,3)
spam(1,2,3, debug=True)

1 2 3
Calling spam
1 2 3


In [12]:
def log_getattribute(cls):
    # Получение изначальной реализации
    orig_getattribute = cls.__getattribute__
    
    # Создание нового определения
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)
    
    # Прикрепление к классу и возврат
    cls.__getattribute__ = new_getattribute
    return cls


# Пример использования
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
    
    def spam(self):
        pass
    
a = A(42)
print(a.x)
a.spam()   

getting: x
42
getting: spam


In [15]:
import time
from contextlib import contextmanager

@contextmanager
def timethis(label):
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print(f'{label}: {end - start}')
        

# Пример
with timethis('counting'):
    n = 10000000
    while n > 0:
        n -= 1

counting: 0.5224344730377197


In [16]:
@contextmanager
def list_transaction(orig_list):
    working = list(orig_list)
    yield working
    orig_list[:] = working
    

items = [1, 2, 3]
with list_transaction(items) as working:
    working.append(4)
    working.append(5)
    
print(items)

[1, 2, 3, 4, 5]


In [18]:
import time

class timethis:
    def __init__(self, label):
        self.label = label
    
    def __enter__(self):
        self.start = time.time()
    
    def __exit__(self, exc_ty, exc_val, exc_tb):
        end = time.time()
        print(f'{self.label}: {end - self.start}')
        

with timethis('counting'):
    n = 10000000
    while n > 0:
        n -= 1

counting: 0.5439958572387695


In [13]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.process_time()
        r = func(*args, **kwargs)
        end = time.process_time()
        print(f'{func.__module__}.{func.__name__}: {end - start}')
        return r
    return wrapper

@timethis
def countdown(n: int):
    '''Counts down'''
    while n > 0:
        n -= 1
        

countdown(100000)
countdown(10000000)

__main__.countdown: 0.0
__main__.countdown: 0.328125


In [5]:
def deco(f):
    print("my_func is running")
    return f

@deco
def my_func(x):
    return x * 2

print(my_func(5))

my_func is running
10


In [2]:
import time 

# Вспомогательный декоратор, он будет декорировать каждый метод класса
def timeit(method):
    def timed(*args, **kwargs):
        start = time.time()
        result = method(*args, **kwargs)
        end = time.time()
        print(f"{method.__name__} выполнялся за {end - start:2.2f} ms")
        return result
    return timed  


def timeit_all_methods(cls):
    class NewCls:
        def __init__(self, *args, **kwargs):
            # Проксируем полностью создание класса
            # Создаем декорируемый класс
            self._obj = cls(*args, **kwargs)
        
        def __getattribute__(self, s):
            try:
                # Есть ли атрибут s?
                x = super().__getattribute__(s)
            except AttributeError:
                print("Такого атрибута нет")
            else:
                # Такой атрибут есть
                return x
            # Если объект, значит, должен быть атрибут 
            attr = self._obj.__getattribute__(s)
            # Метод ли он?
            if isinstance(attr, type(self.__init__)):
                # Да, обернуть его в измеритель времени
                return timeit(attr)
            else:
                # Не метод, что-то другое
                return attr 
    return NewCls

@timeit_all_methods
class Foo:
    def a(self):
        print("метод а начинает работу")
        time.sleep(0.888)
        print("метод а завершает работу")
        

f = Foo()
f.a()
        

Такого атрибута нет
метод а начинает работу
метод а завершает работу
a выполнялся за 0.90 ms


In [21]:
from rich import print

def wrap(color):
    def wrapper(func):
        def gift_wrap(*args, **kwargs):
            print(f"[{color}]*обертка*[/{color}]")
            func(*args, **kwargs)
            print(f"[{color}]*обертка*[/{color}]")
        return gift_wrap
    return wrapper

@wrap("purple")
def gift(x):
    print(x)
    
gift('[green]ПОДАРОК[/green]')

*обертка*

ПОДАРОК

*обертка*

In [24]:
from rich import print

class Silent:
    def __init__(self, func):
        self.func = func 
    
    def __call__(self, *args, **kwargs):
        print("[green]*прикрутил глушитель*[/green]")
        self.func(*args, **kwargs)
        
@Silent
def usp(aim):
    print(f"Пиу-Пау [purple]{aim}[/purple] :zany_face:")
    
usp("magwindow")

*прикрутил глушитель*

Пиу-Пау magwindow 🤪

In [29]:
from rich import print

class Silent:
    def __init__(self, shot_range):
        self.shot_range = shot_range 
    
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print("[green]*прикрутил глушитель*[/green]")
            func(*args, **kwargs)
            print(f"Дистанция: [green]{self.shot_range}m[/green]")
        return wrapper
        
@Silent(120)
def usp(aim):
    print(f"Пиу-Пау [purple]{aim}[/purple] :zany_face:")
    
usp("magwindow")

*прикрутил глушитель*

Пиу-Пау magwindow 🤪

Дистанция: 120m

In [31]:
def wrap(cls):
    def wrapper(*args, **kwargs):
        obj = cls(*args, **kwargs)
        # obj.something = "TEST"
        obj.some_action()
    return wrapper

@wrap
class SomeClass:
    def __init__(self, something):
        self.something = something
        
    def some_action(self):
        print(self.something)

SomeClass("Message")

Message

In [34]:
def wrap(text):
    def wrapper(function):
        function(text)
    return wrapper 

@wrap("Hello, World!")
def function(some_text):
    print(some_text)

Hello, World!